## Import Libraries

In [3]:
import pandas as pd
import regex as re
import numpy as np
import logging

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import mplcyberpunk
%config InlineBackend.figure_formats = ['svg']

# ML Modeling
from sklearn.metrics import precision_recall_fscore_support,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import RocCurveDisplay,roc_curve
from sklearn.preprocessing import normalize
# Saving and importing trained models
import pickle

Read through Cleaned Dataset

In [13]:
df = pd.read_csv("data/processed-dataset.csv")

hispanic_filter = df['race'] == "HISPANIC/LATIN/MEXICAN"
hispanic_df = df[hispanic_filter].sort_values(by = ['arrest_date', 'arrest_time'], ascending= [True, True])
hispanic_df.head(10)

/var/folders/rj/l_96jjr56y39j9jw2hnmv_wh0000gn/T/ipykernel_1826/1538011803.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/processed-dataset.csv")


,arrest_unique_id,incident_number,pin,age,sex,race,ethnicity,arrest_date,arrest_time,location_of_arrest_in_block,arrest_reason,arrest_type,summary_of_facts,arrest_officer,officer_name,current_status,young_offender
56511,SJ201519,SJ2015150010241,441762891,19.0,M,HISPANIC/LATIN/MEXICAN,HISPANIC/LATIN/MEXICAN,01/01/15,415.0,2900 BLOCK OCALA CT,CRIMINAL CITATION,TAKEN INTO CUSTODY/WARRANT ARREST ONLY,TRESPASSING (CITE),4222,MACIAS,CITED,N
56522,SJ201519,SJ2015J3347648 C,441762891,19.0,M,HISPANIC/LATIN/MEXICAN,HISPANIC/LATIN/MEXICAN,01/01/15,415.0,2900 BLOCK OCALA CT,CRIMINAL CITATION,TAKEN INTO CUSTODY/WARRANT ARREST ONLY,TRESPASSING (CITE),4222,MACIAS,CITED,N
51970,SJ20157,SJ2015150010272,441776037,23.0,F,HISPANIC/LATIN/MEXICAN,HISPANIC/LATIN/MEXICAN,01/01/15,530.0,5100 BLOCK MONTEREY RD,ON VIEW,ON VIEW ARREST,INFLICT CORP INJ ON SPOUSE/COHAB (ONVW),4348,LEE,CHARGED/BOOKED,N
94333,SJ2015190,SJ2015150010482,441915772,33.0,F,HISPANIC/LATIN/MEXICAN,HISPANIC/LATIN/MEXICAN,01/01/15,1400.0,2200 BLOCK TULLY RD,CRIMINAL CITATION,TAKEN INTO CUSTODY/WARRANT ARREST ONLY,PETTY THEFT OF PERSONAL PROPERTY/LABOR/EMBEZZL...,3153,TOMPKINS,CITED,N
94339,SJ2015190,SJ2015J3337366 C,441915772,33.0,F,HISPANIC/LATIN/MEXICAN,HISPANIC/LATIN/MEXICAN,01/01/15,1400.0,2200 BLOCK TULLY RD,CRIMINAL CITATION,TAKEN INTO CUSTODY/WARRANT ARREST ONLY,PETTY THEFT OF PERSONAL PROPERTY/LABOR/EMBEZZL...,3153,TOMPKINS,CITED,N
3262,SJ2015337,SJ2015J3354478 C,441916123,23.0,M,HISPANIC/LATIN/MEXICAN,HISPANIC/LATIN/MEXICAN,01/01/15,1605.0,CURTNER AV / 87,CRIMINAL CITATION,TAKEN INTO CUSTODY/WARRANT ARREST ONLY,FAILING TO PROVIDE EVIDENCE OF FINANCIAL RESPO...,4280,ARANA,CITED,N
96726,SJ201529,SJ2015150010655,441915179,22.0,M,HISPANIC/LATIN/MEXICAN,HISPANIC/LATIN/MEXICAN,01/01/15,1655.0,S 1ST ST / W ALMA AV,ON VIEW,ON VIEW ARREST,"RESISTING, DELAYING, OBSTRUCTING AN OFFICER (O...",4245,VALOSEK,CHARGED/BOOKED,N
14755,SJ201525,SJ2015150010664,51315201,59.0,M,HISPANIC/LATIN/MEXICAN,HISPANIC/LATIN/MEXICAN,01/01/15,1756.0,MERIDIAN AV / DOUGLAS ST,LOCAL BENCH WARRANT,TAKEN INTO CUSTODY/WARRANT ARREST ONLY,POSSESSION OF CONTROLLED SUBSTANCE PARAPHERNAL...,4152,BYERS,CHARGED/BOOKED,N
91122,SJ201530,SJ2015150010664,441729075,59.0,M,HISPANIC/LATIN/MEXICAN,HISPANIC/LATIN/MEXICAN,01/01/15,1756.0,300 BLOCK MERIDIAN AV,ON VIEW,ON VIEW ARREST,POSSESSION CONTROLLED SUBSTANCE (ONVW),3773,SOLOMON,CHARGED/BOOKED,N
65769,SJ201514,SJ2015150010728,441761283,23.0,M,HISPANIC/LATIN/MEXICAN,HISPANIC/LATIN/MEXICAN,01/01/15,1831.0,5600 BLOCK COTTLE RD,ON VIEW,ON VIEW ARREST,ROBBERY (ONVW),4240,MORGAN II,CHARGED/BOOKED,N
